In [ ]:
import sys
sys.path.append('../implementation/')
from gotz_adaptive_contextualization import AC
import pandas as pd
import ast
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from util import flatten_list
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Loading the underlying data and user interaction data
data_path = '../data/vast_2011_challenge/vast_data_sample_reduced.pkl'
bookmarks_ui_data_path = '../data/vast_2011_challenge/bookmark_interactions_clean.pkl'
hovers_ui_data_path = '../data/vast_2011_challenge/hover_interactions.pkl'
hovers = False
if hovers:
    output_file_path = '../output/vast/vast_11_hovers_ac.pkl'
    interaction_data = pd.read_pickle(hovers_ui_data_path)
else:
    output_file_path = '../output/vast/vast_11_ac.pkl'
    interaction_data = pd.read_pickle(bookmarks_ui_data_path)
    interaction_data = interaction_data[interaction_data['experimental_group'] == 'control']
    interaction_data = interaction_data.reset_index(drop=True)

underlying_data = pd.read_pickle(data_path)

ks = [1, 5, 10, 20, 50, 100]
d_attributes = ['topic']
c_attributes = [['latitude', 'longitude']]

In [ ]:
ac_results = pd.DataFrame()
for participant_index, row in interaction_data.iterrows():
    print(f'Processing user {row.user} task {row.experimental_group}')
    results = {'participant_id': row.user, 'task': row.experimental_group}
    ac_model = AC(underlying_data, c_attributes, d_attributes)
    for i in tqdm(range(len(interaction_data.iloc[participant_index].interaction_session))):
        interaction = interaction_data.iloc[participant_index].interaction_session[i]
        ac_model.update(interaction)

    bias = ac_model.get_attribute_bias()
    for col in bias.columns:
        results[f'bias-{col}'] = bias[col].to_numpy()
    if 'latitude_disc' in bias.columns:
        results['bias-latitude___longitude'] = results['bias-latitude_disc'] * results['bias-longitude_disc']
    
    ac_results = ac_results.append(results, ignore_index=True)
    
ac_results.to_pickle(output_file_path)